# Data Loading and preprocessing

In [1]:
# Import required packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set the style for better looking plots
plt.style.use('ggplot')
sns.set(font_scale=1.2)


In [2]:
# Load the dataset
df = pd.read_csv('medical_claims_dataset.csv')
df.head()

,claim_id,patient_id,patient_age,patient_gender,specialty,provider_id,procedure_code,diagnosis_code,service_date,claim_submission_date,claim_amount,is_denied,claim_status,denial_reason,denial_category,insurance_provider
0,4754401,517495,32,Male,Dermatology,72808,99214,E11.9,2024-12-11,2025-02-11,5214.51,False,Approved,NaN,NaN,Blue Cross Blue Shield
1,1467150,7037,72,Male,Plastic Surgery,98858,99213,E78.5,2025-02-08,2025-05-01,1424.14,False,Approved,NaN,NaN,Anthem
2,1297566,334530,71,Male,Infectious Diseases,82385,99214,F32.9,2023-07-24,2025-01-31,37964.49,False,Approved,NaN,NaN,UnitedHealthcare
3,5198295,946957,53,Male,General Surgery,68703,99212,E11.9,2025-05-25,2025-06-02,19058.37,False,Approved,NaN,NaN,Humana
4,5715757,629227,51,Male,Dermatology,51177,99215,R10.9,2024-04-18,2024-09-11,18988.87,False,Approved,NaN,NaN,Blue Cross Blue Shield


In [3]:
# Display basic information
print(f"Dataset shape: {df.shape}")
print("\nFirst 5 rows:")
print(df.head())


Dataset shape: (10000, 16)

First 5 rows:
   claim_id  patient_id  patient_age patient_gender            specialty  \
0   4754401      517495           32           Male          Dermatology   
1   1467150        7037           72           Male      Plastic Surgery   
2   1297566      334530           71           Male  Infectious Diseases   
3   5198295      946957           53           Male      General Surgery   
4   5715757      629227           51           Male          Dermatology   

   provider_id  procedure_code diagnosis_code service_date  \
0        72808           99214          E11.9   2024-12-11   
1        98858           99213          E78.5   2025-02-08   
2        82385           99214          F32.9   2023-07-24   
3        68703           99212          E11.9   2025-05-25   
4        51177           99215          R10.9   2024-04-18   

  claim_submission_date  claim_amount  is_denied claim_status denial_reason  \
0            2025-02-11       5214.51      False 

In [4]:
# Check for missing values
print("\nMissing values per column:")
print(df.isnull().sum())




Missing values per column:
claim_id                    0
patient_id                  0
patient_age                 0
patient_gender              0
specialty                   0
provider_id                 0
procedure_code              0
diagnosis_code              0
service_date                0
claim_submission_date       0
claim_amount                0
is_denied                   0
claim_status                0
denial_reason            7989
denial_category          7989
insurance_provider          0
dtype: int64


In [5]:
# Handle missing values
# For denied claims, DenialReason and DenialCategory should be present
# For approved claims, we'll fill with 'Not Applicable'
df.loc[df['claim_status'] == 'Approved', 'denial_reason'] = 'Not Applicable'
df.loc[df['claim_status'] == 'Approved', 'denial_category'] = 'Not Applicable'


In [6]:
# Convert dates to datetime objects
df['service_date'] = pd.to_datetime(df['service_date'])
df['claim_submission_date'] = pd.to_datetime(df['claim_submission_date'])

In [7]:
# Create new features from dates
df['processing_days'] = (df['claim_submission_date'] - df['service_date']).dt.days
df['Month'] = df['service_date'].dt.month
df['WeekDay'] = df['service_date'].dt.dayofweek


In [8]:
# Check if any missing values remain
print("Missing values after preprocessing:")
print(df.isnull().sum())

Missing values after preprocessing:
claim_id                 0
patient_id               0
patient_age              0
patient_gender           0
specialty                0
provider_id              0
procedure_code           0
diagnosis_code           0
service_date             0
claim_submission_date    0
claim_amount             0
is_denied                0
claim_status             0
denial_reason            0
denial_category          0
insurance_provider       0
processing_days          0
Month                    0
WeekDay                  0
dtype: int64


# EDA

In [9]:
# Visualize denied vs approved claims
# Number of claims
status_counts = df['claim_status'].value_counts()
status_counts

claim_status
Approved    7989
Denied      2011
Name: count, dtype: int64

In [10]:
# Pie chart
plt.figure(figsize=(6, 6))
plt.pie(status_counts, labels=status_counts.index, autopct="%1.01f%%", colors=["lightblue", "pink"], startangle=90)
plt.title("Claims Approval vs Denial Rate")
plt.savefig('static/images/claim_status_distribution.png')
plt.close()
plt.show()
# # Bar chart
# plt.figure(figsize=(6, 4))
# plt.bar(status_counts.index, status_counts.values, color=["lightblue", "pink"])
# plt.xlabel("Claim Status")
# plt.ylabel("Number of Claims")
# plt.title("Number of Approved vs Denied Claims")
# plt.show()

In [11]:
# Denied claims dataframe
denied_claims = df[df['claim_status']=='Denied']
denied_claims.head()

,claim_id,patient_id,patient_age,patient_gender,specialty,provider_id,procedure_code,diagnosis_code,service_date,claim_submission_date,claim_amount,is_denied,claim_status,denial_reason,denial_category,insurance_provider,processing_days,Month,WeekDay
11,8399858,755286,69,Female,General Surgery,54242,99202,E78.5,2023-07-24,2024-12-09,4636.38,True,Denied,Missing Medical Necessity Justification,Middle Office,UnitedHealthcare,504,7,0
13,7867026,945025,19,Other,Gastroenterology,10753,99204,E78.5,2024-10-07,2024-10-23,13414.58,True,Denied,Incomplete Registration Data,Front Office,Blue Cross Blue Shield,16,10,0
25,9932277,511798,27,Female,Cardiology,86498,99211,E11.9,2025-05-26,2025-06-01,32642.03,True,Denied,Eligibility Verification Failure,Front Office,Blue Cross Blue Shield,6,5,0
26,334971,145465,48,Male,Neurology,91274,99203,F32.9,2023-06-16,2023-07-26,45131.90,True,Denied,Incorrect Insurance Information,Front Office,UnitedHealthcare,40,6,4
27,9067049,900785,76,Female,Neurology,42080,90471,R10.9,2025-03-03,2025-04-12,27832.84,True,Denied,Incorrect Insurance Information,Front Office,Blue Cross Blue Shield,40,3,0


In [12]:
# Percentage of denial category
denial_reasons_counts = denied_claims['denial_category'].value_counts()
denial_reasons_counts

# Piechart to visualize the denial category percentages
plt.figure(figsize=(8,8))
denial_reasons_counts.plot(kind="pie", autopct='%1.1f%%', colormap="Set3", startangle=90)
plt.title("Breakdown of claim denial by opearional area")
plt.ylabel('')
plt.savefig('static/images/claimdenialcategory.png')
plt.close()
plt.show()

In [13]:
# Distribution of denial reasons
plt.figure(figsize=(12, 6))
sns.barplot(x=denied_claims['denial_reason'].value_counts().index, y=denied_claims['denial_reason'].value_counts().values, palette="pastel")
plt.xticks(rotation=45, ha='right')
plt.xlabel('Denial Reasons')
plt.ylabel('Count')
plt.title('Distribution of Denial Reasons')
plt.savefig('static/Images/denialreasons.png', bbox_inches='tight')
plt.close()
plt.show()



In [21]:
# Calculate specialty percentages
specialty_percentages = df['specialty'].value_counts(normalize=True)*100

# Create horizontal bar chart
plt.figure(figsize=(12, 8))
specialty_percentages.sort_values().plot(kind='barh')
plt.title('Distribution of claims by medical specialty', fontsize=16)
plt.xlabel('Percentage of Claims', fontsize=12)
plt.ylabel('Specialty', fontsize=12)
plt.savefig('static/Images/specialty.png', bbox_inches='tight')
plt.close()

# # Create pie chart of top 10 specialties
# plt.figure(figsize=(12, 8))
# top_10_specialties = specialty_percentages.head(10)
# plt.pie(top_10_specialties, labels=top_10_specialties.index, autopct='%1.1f%%')
# plt.title('Top 10 Medical Specialties Distribution', fontsize=16)
# plt.axis('equal')
# plt.savefig('specialty.png')
# plt.show()